# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, split, explode, regexp_replace, transform

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/22 19:52:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

25/03/22 19:52:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [3]:
# Your Code for 1st question here
# Filter the dataset
df = df.filter((col("SALARY_FROM").isNotNull()) & (col("SALARY_FROM") > 0))

# Aggregate Data
pdf = df.select("EMPLOYMENT_TYPE_NAME", "SALARY").toPandas()

# Visualize results
fig = px.box(pdf, x="EMPLOYMENT_TYPE_NAME", y="SALARY", title="Salary Distribution by Employment Type", 
             color_discrete_sequence=["#66CCFF"])
fig.update_layout(font_family="Arial", title_font_size=30, title_x=0.5)
fig.show()

fig.write_image("output/Q1.svg", width=1920, height=540, scale=2)

![Question 1](output/Q1.svg)

The graph reveals that full-time employees (> 32 hours) generally have higher median salaries compared to part-time employees (≤ 32 hours), as indicated by the higher position of the box plots for full-time employment. Additionally, the salary distribution for full-time employees shows a wider range and more variability, suggesting greater disparity in earnings within this group compared to part-time employees.

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [4]:
# Your code for 2nd question here
# Filter the dataset
df = df.filter(col("SALARY_FROM") > 0)

# Aggregate Data
pdf = df.select("NAICS2_NAME", "SALARY_FROM").toPandas()

# Visualize results
fig = px.box(pdf, x="NAICS2_NAME", y="SALARY_FROM", title="Salary Distribution by Employment Type", 
    color_discrete_sequence=["#66CCFF"])

fig.update_layout(font_family="Arial", title_font_size=30, title_x=0.5,
                  xaxis_title="Employment Type",  yaxis_title="Salary From ($)",
                  xaxis=dict(tickangle=45, showgrid=True, gridcolor="grey"),
                  yaxis=dict(showgrid=True, gridcolor="grey"),
                  legend=dict(title="Employment Type",  font=dict(family="Arial", size=14, color="black"), 
                  bgcolor="white",  bordercolor="black", borderwidth=1))

fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1)), boxmean=True, line=dict(width=2))
fig.show()

fig.write_image("output/Q2.svg", width=1920, height=1080, scale=2)

![Question 2](output/Q2.svg)

The chart illustrates the distribution of salaries across industries and types of employment. The chart shows significant differences in median pay across industries, with higher median pay in industries such as information compared to industries such as education services.

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [5]:
# Your code for 3rd question here
# Aggregate Data
df_aggregated = df.groupBy("POSTED").count()
pdf = df_aggregated.toPandas()

# Visualize results
fig = px.line(pdf, x="POSTED", y="count", title="Job Posting Trends Over Time", line_shape="linear",
              color_discrete_sequence=["#66CCFF"])

fig.update_layout(font_family="Arial", title_font_size=30, title_x=0.5, 
                  xaxis_title="Posted Date", yaxis_title="Number of Job Postings")

fig.show()

fig.write_image("output/Q3.svg", width=1920, height=540, scale=2)

![Question 3](output/Q3.svg)

The graph reveals that the number of job postings fluctuates significantly over time, with noticeable peaks and troughs indicating periods of high and low job posting activity. These fluctuations could be influenced by seasonal hiring trends, economic conditions, or specific industry demands.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [6]:
# Your code for 4th question here
# Aggregate Data
df_aggregated = df.groupBy("TITLE_NAME").count()
df_top_10 = df_aggregated.orderBy(col("count").desc()).limit(10)
pdf = df_top_10.toPandas()

# Visualize results
fig = px.bar(pdf, x="TITLE_NAME", y="count", title="Top 10 Job Titles by Count", 
             color="count", color_continuous_scale="Blues")

fig.update_layout(font_family="Arial", title_font_size=30, title_x=0.5, 
                  xaxis_title="Job Title", yaxis_title="Job Count")

fig.show()

fig.write_image("output/Q4.svg", width=1920, height=540, scale=2)

![Question 4](output/Q4.svg)

The graph reveals that "Data Analysts" is the most frequently posted job title, significantly outpacing other roles in terms of job count. Other roles like "Business Intelligence Analysts" and "Data Analytics Engineers" also appear frequently, indicating a strong demand for data-related positions in the job market.

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [7]:
# Your code for 5th question here
# Aggregate Data
df_aggregated = df.groupBy("REMOTE_TYPE_NAME").count()
pdf = df_aggregated.toPandas()

# Visualize results
fig = px.pie(pdf, names="REMOTE_TYPE_NAME", values="count", title="Remote vs On-Site Job Postings",
             color="REMOTE_TYPE_NAME", color_discrete_map={"Remote": "#66CCFF", "On-site": "#EE0000"})

fig.update_layout(font_family="Arial", title_font_size=30,  title_x=0.5)

fig.show()

fig.write_image("output/Q5.svg", width=1080, height=540, scale=2)

![Question 5](output/Q5.svg)

The graph reveals that the majority of job postings (73.2%) do not specify a remote type, indicating a lack of clarity or preference in remote work options. Among the specified postings, remote jobs make up 21.3%, while hybrid remote and not remote jobs account for smaller proportions at 3.55% and 1.94%, respectively.

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [10]:
# Your code for 6th question here
# Aggregate Data
df_split = df.withColumn("SKILLS_NAME", split(col("SKILLS_NAME"), ",\n  "))
df_split = df_split.withColumn("SKILLS_NAME",
    transform(col("SKILLS_NAME"), lambda x: regexp_replace(x, r'\n', '')))
df_exploded = df_split.withColumn("Skill", explode(col("SKILLS_NAME")))
df_aggregated = df_exploded.groupBy("NAICS2_NAME", "Skill").count()
pdf = df_aggregated.toPandas()

skill_count = pdf.groupby('Skill')['count'].sum().sort_values(ascending=False)
top_skills = skill_count.head(10).index
pdf_filtered = pdf[pdf['Skill'].isin(top_skills)]
pdf_filtered = pdf_filtered.dropna(subset=['Skill'])
pdf_filtered['Skill'] = pdf_filtered['Skill'].astype(str)

# Visualize results
fig = px.bar(pdf_filtered, x='NAICS2_NAME', y='count', color='Skill',
             title='Stacked Bar Chart: Industry and Skill Distribution',
             labels={'NAICS2_NAME': 'Industry', 'count': 'Skill Count', 'Skill': 'Skill'},
             barmode='stack', color_discrete_sequence=px.colors.sequential.Viridis)

fig.update_layout(title_font_size=16, xaxis_title_font_size=14, yaxis_title_font_size=14,
    legend_title_font_size=12, xaxis_tickangle=45)
fig.show()

fig.write_image('output/Q6.svg', width=1920, height=1080, scale=2)


![Question 6](output/Q6.svg)

The graph reveals that "Data Analysis" is the most in-demand skill across various industries, with the highest skill count observed in the "Professional, Scientific, and Technical Services" sector. Additionally, skills like "Microsoft Excel," "Project Management," and "SQL (Programming Language)" are also highly sought after, particularly in industries such as "Finance and Insurance" and "Health Care and Social Assistance."


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [11]:
# Your code for 7th question here
# Aggregate Data
df_aggregated = df.groupBy("ONET_NAME").agg(
    F.expr("percentile_approx(SALARY_FROM, 0.5)").alias("Median_Salary"),
    F.count("ID").alias("Job_Postings"))
df_aggregated_pd = df_aggregated.toPandas()

# Visualize results
fig = px.scatter(df_aggregated_pd, x="ONET_NAME", y="Median_Salary",
                 size="Job_Postings", color="ONET_NAME", title="Salary Analysis by ONET Occupation Type",
                 labels={'ONET_NAME': 'ONET Occupation Type', 'Median_Salary': 'Median Salary'},
                 color_continuous_scale="Viridis")

fig.update_layout(font_family="Arial", showlegend=True, title_font_size=30, title_x=0.5, 
                  xaxis_title="ONET Occupation Type", yaxis_title="Median Salary",
                  margin=dict(l=50, r=50, t=100, b=50))
fig.show()

fig.write_image("output/Q7.svg", width=1080, height=540, scale=2)

![Question 7](output/Q7.svg)

The graph reveals that the median salary for "Business Intelligence Analysts" is approximately $88,000, as indicated by the position of the bubble on the y-axis. The size of the bubble suggests the number of job postings for this occupation, providing insight into both salary levels and demand within this specific ONET occupation type.

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [12]:
# Your code for 8th question here
# Aggregate Data
df_transitions = df.groupBy("SOC_2021_2_NAME", "SOC_2021_3_NAME").count()
df_transitions_pd = df_transitions.toPandas()

# Visualize results
unique_sources = df_transitions_pd["SOC_2021_2_NAME"].unique()
unique_targets = df_transitions_pd["SOC_2021_3_NAME"].unique()
nodes = list(unique_sources) + list(unique_targets)
node_indices = {node: i for i, node in enumerate(nodes)}
links = {
    "source": [node_indices[source] for source in df_transitions_pd["SOC_2021_2_NAME"]],
    "target": [node_indices[target] for target in df_transitions_pd["SOC_2021_3_NAME"]],
    "value": df_transitions_pd["count"].tolist()
}

fig = go.Figure(go.Sankey(
    node=dict(pad=15, thickness=20, line=dict(color="black", width=0.5), label=nodes, color="blue"),
    link=dict(source=links["source"], target=links["target"], value=links["value"], color="green")
))

fig.update_layout(title="Career Pathway Trends", font_family="Arial", title_font_size=30, title_x=0.5,
                  plot_bgcolor="white", paper_bgcolor="white", 
                  xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
fig.show()

fig.write_image("output/Q8.svg", width=1080, height=540, scale=2)

![Question 8](output/Q8.svg)

The graph reveals a clear transition from "Computer and Mathematical Occupations" to "Mathematical Science Occupations," indicating a common career pathway within these fields. The width of the link suggests the volume of transitions, highlighting the significance of this career progression.